In [17]:
import pandas as pd
import numpy as np

In [2]:
path_ = '../data/db_bsm_financial.csv'

In [72]:
data_ = pd.read_csv(path_)
data_['date'] = pd.to_datetime(data_['date'])
data_filt = data_[['ticker', 'date', 'close', 'volume']].sort_values(['ticker', 'date'], ascending=[True, True]).reset_index(drop=True)

In [73]:
def add_pl(df_, col_id, col_calc_pl):
    for id_ in df_[col_id].unique():
        df_by_id = df_[df_[col_id] == id_]
        df_.loc[df_[col_id] == id_, 'daily_pl'] = df_by_id[col_calc_pl] - df_by_id[col_calc_pl].shift(1)
    return df_

data_filt = add_pl(data_filt, 'ticker', 'close')

In [74]:
data_filt.isna().sum()

ticker      0
date        0
close       0
volume      0
daily_pl    4
dtype: int64

In [76]:
data_filt = data_filt.dropna()

In [77]:
data_filt.isna().sum()

ticker      0
date        0
close       0
volume      0
daily_pl    0
dtype: int64